In [3]:
%matplotlib inline
import pathlib
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 8, 6
import pandas as pd
import numpy as np
import seaborn as sns
pd.set_option('display.max_columns', 500)
from collections import defaultdict

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

pd.options.mode.chained_assignment = None

import glob
from tqdm import tqdm, tqdm_notebook, tnrange
tqdm.pandas(desc='Progress')

from market import *
from market_ml import *

C:\Users\kevin\Anaconda3\envs\tensorflow_env\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
def load_csv_data():
    csv_filenames = glob.glob("csv_files/company_stats_*.csv")
    financial_data = pd.DataFrame()
    for csv_name in tqdm(csv_filenames):
        # Remove rows with nan values for industry and sector
        try:
            daily_data = pd.read_csv(csv_name, encoding='cp1252')
            daily_data = daily_data[pd.notnull(daily_data['Industry'])] 
            daily_data = daily_data[pd.notnull(daily_data['Sector'])]
            financial_data = financial_data.append(daily_data, sort=False)
        except:
            print('Importing data from', csv_name, 'failed.')

    financial_data = financial_data.reset_index()
    print('Master dataframe columns: ', financial_data.columns)

    # Convert sector and industry to categoricals
    #financial_data['Industry'] = financial_data['Industry'].astype('category')
    #financial_data['Sector'] = financial_data['Sector'].astype('category')

    #to_remove = ['Ticker', 'Name', 'Price', 'IPO Year']
    #categorical = ['Sector', 'Industry']

    # Convert categorical variables to integers to feed in the model
    #financial_data[categorical] = financial_data[categorical].apply(lambda x: x.cat.codes)
    #feature_cols = [x for x in financial_data.columns if x not in to_remove]
    #financial_data = financial_data.fillna(0)
    #X = financial_data[feature_cols]
    #Y = financial_data['Price']
    
    return financial_data


financial_data = load_csv_data()

 75%|███████▍  | 164/219 [00:58<00:43,  1.28it/s]

Importing data from csv_files\company_stats_2020-04-09.csv failed.


 86%|████████▌ | 188/219 [01:18<00:28,  1.11it/s]

Importing data from csv_files\company_stats_2020-05-17.csv failed.


100%|██████████| 219/219 [01:48<00:00,  2.01it/s]


Master dataframe columns:  Index(['index', 'Ticker', 'Name', 'Sector', 'Industry', 'IPO Year', 'Price',
       'Market Cap', 'Trailing P/E', 'Forward P/E', 'PEG Ratio(5yr Expected)',
       'Price/Sales(ttm)', 'Price/Book', 'Enterprise Value/Revenue',
       'Enterprise Value/EBITDA', 'Profit Margin', 'Operating Margin(TTM)',
       'Return on Assets(TTM)', 'Return on Equity(TTM)', 'Revenue(TTM)',
       'Revenue Per Share(TTM)', 'Quarterly Revenue Growth(YOY)',
       'Gross Profit(TTM)', 'EBITDA', 'Diluted EPS(TTM)',
       'Quarterly Earnings Growth(YOY)', 'Total Cash', 'Total Cash Per Share',
       'Total Debt', 'Total Debt/Equity', 'Current Ratio',
       'Book Value Per Share', 'Operating Cash Flow(TTM)',
       'Levered Free Cash Flow(TTM)', 'Beta(3Y Monthly)', 'Shares Outstanding',
       'Forward Annual Dividend Rate', 'Forward Annual Dividend Yield',
       'Trailing Annual Dividend Rate', 'Trailing Annual Dividend Yield',
       '5 Year Average Dividend Yield', 'Payout Rati

In [9]:
def parse_col_attributes(col_name):
    replacements = {
                        '(TTM)' : '',
                        '(ttm)' : '',
                        '(YOY)' : '',
                        ' (intraday)' : '',
                        ' (5 yr expected)' : '',
                        '(5yr Expected)': '',
                        '(3Y Monthly)' : '',
                        ' (3Y Monthly)' : '',
                        ' Avi to Common' : '',
                        'Trailing Annual ' : ''
                   }
    for code, replacement in replacements.items():
        col_name = col_name.replace(code, replacement)
    return col_name

def preprocess_data(financial_data):
    # Get the unique column values in the dataframe so we can combine data
    unique_attributes = set([parse_col_attributes(col) for col in financial_data.columns])

    # Iterate through attributes and fill master_data
    master_data = pd.DataFrame()
    for attribute in tqdm(unique_attributes):
        supporting_attributes = [x for x in financial_data.columns if attribute in x]
        master_data[attribute] = financial_data[supporting_attributes[0]]
        # Combine columns with same attribute using the non zero value
        for i in range(1, len(supporting_attributes)):
            master_data[attribute] = master_data[attribute].combine(financial_data[supporting_attributes[i]], lambda x, y: x if (y == 0 or float('nan')) else y)

    # Columns to ignore when training
    ignore_columns = ['Price', 'Ticker', 'Name', 'IPO Year', 'Payout Ratio', 'Enterprise Value/EBITDA', 'Enterprise Value/Revenue',
                     'Trailing Annual Dividend Rate', 'Forward Annual Dividend Rate', 'Forward Annual Dividend Yield',
                     '5 Year Average Dividend Yield', 'EPS Beat Ratio', 'Beta ', 'index']
    # Keep track of categoricals so we can convert them to numeric
    categoricals = ['Sector', 'Industry']

    # Store the data we want to learn from in X
    X = master_data[[c for c in master_data.columns if c not in ignore_columns]]
    Y = master_data['Price']

    # Convert categoricals to integers
    for c in categoricals:
        financial_data[c] = financial_data[c].astype('category')
    financial_data[categoricals] = financial_data[categoricals].apply(lambda x: x.cat.codes)
    
    for c in X.columns:
        X[c] = pd.to_numeric(X[c], errors='coerce')
    #X = pd.get_dummies(X, columns=categoricals)

    # Get training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=123)
    print('Columns of training data:', X_train.columns)
    print('Training Size:', len(X_train))
    
    # Normalize the non categorical columns
    #non_categoricals = list(set(list(X.columns)).difference(set(categoricals)))
    #X_train[non_categoricals]=(X_train[non_categoricals]-X_train[non_categoricals].min())/(X_train[non_categoricals].max()-X_train[non_categoricals].min())
    #X_test[non_categoricals]=(X_test[non_categoricals]-X_test[non_categoricals].min())/(X_test[non_categoricals].max()-X_test[non_categoricals].min())

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = preprocess_data(financial_data)

100%|██████████| 46/46 [09:07<00:00, 11.90s/it]


Columns of training data: Index(['Return on Equity', 'Price/Book', 'Dividend Rate', 'Industry',
       'Quarterly Earnings Growth', 'Revenue', 'Quarterly Revenue Growth',
       'Beta', 'Market Cap', 'Diluted EPS', 'Levered Free Cash Flow', 'Sector',
       'Revenue Per Share', 'Forward P/E', 'Current Ratio', 'Net Income',
       'Total Debt', 'EBITDA', 'Price/Sales', 'Gross Profit', 'Profit Margin',
       'Operating Cash Flow', 'Dividend Yield', 'Total Cash',
       'Book Value Per Share', 'PEG Ratio', 'Total Cash Per Share',
       'Total Debt/Equity', 'Operating Margin', 'Return on Assets',
       'Enterprise Value', 'Trailing P/E', 'Shares Outstanding'],
      dtype='object')
Training Size: 586670


In [10]:
# Hyperparameter tune settings for XGBoost model
param_test = {
    'max_depth':[3],
    'min_child_weight':[4],
    'learning_rate':[.25],
    'gamma':[0],
    'reg_alpha':[ 0.1]#, .12, .14]
}

# Define XGB model object
xgbr = xgb.XGBRegressor(objective='reg:squarederror')

# Define grid search object and feed in the model we created
gsearch = GridSearchCV(estimator=xgbr, param_grid=param_test,n_jobs=4,iid=False, cv=5)

# Fit perform the grid search
samp_size = 586670 / 10
gsearch.fit(X_train, y_train)
print("Test Score: " + str(gsearch.best_estimator_.score(X_test, y_test)))
model = gsearch.best_estimator_
preds = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
model.save_model('xgb_main.model')

C:\Users\kevin\Anaconda3\envs\tensorflow_env\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:squarederror',
                                    random_state=0, reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid=False, n_jobs=4,
             param_grid={'gamma': [0], 'learning_rate': [0.25],
                         'max_depth': [3], 'min_child_we

Test Score: 0.8737420944473197
RMSE: 21.876113


In [11]:
X_train.head()

,Return on Equity,Price/Book,Dividend Rate,Industry,Quarterly Earnings Growth,Revenue,Quarterly Revenue Growth,Beta,Market Cap,Diluted EPS,Levered Free Cash Flow,Sector,Revenue Per Share,Forward P/E,Current Ratio,Net Income,Total Debt,EBITDA,Price/Sales,Gross Profit,Profit Margin,Operating Cash Flow,Dividend Yield,Total Cash,Book Value Per Share,PEG Ratio,Total Cash Per Share,Total Debt/Equity,Operating Margin,Return on Assets,Enterprise Value,Trailing P/E,Shares Outstanding
163313,NaN,1.61,0.08,177,NaN,2.41,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1.61,88900000.0,2.330000e+09,6.21,NaN,NaN,3.41,NaN,1.54,534800000.0,3.25,NaN,0.64,82.43,NaN,NaN,2.41,51.98,828630000.0
28077,0.21,0.73,NaN,247,NaN,1.90,1.8,0.97,6.041540e+08,0.01,-38070000.0,24,8.70,3.01,1.34,NaN,7.243000e+08,8.11,0.91,169400000.0,0.27,120740000.0,NaN,106400000.0,11.70,NaN,1.44,83.80,6.84,1.63,1.90,566.67,73900000.0
62601,6.72,0.98,NaN,240,NaN,2.57,25.1,1.14,6.626310e+08,0.72,NaN,12,16.82,12.63,NaN,NaN,6.351200e+08,NaN,1.46,436440000.0,4.36,142010000.0,NaN,127550000.0,27.75,3.08,5.23,NaN,14.69,0.67,2.57,37.83,24400000.0
29787,76.14,22.08,2.20,203,12.3,2.70,6.0,0.91,7.582000e+09,6.07,148940000.0,7,76.96,28.14,2.35,NaN,8.682900e+08,23.23,2.47,870170000.0,8.14,252910000.0,1.17,61410000.0,8.60,1.81,1.54,253.02,10.71,13.30,2.70,31.31,39920000.0
30552,-18.37,10.79,NaN,50,NaN,NaN,-17.1,1.14,4.943000e+09,-1.73,NaN,15,NaN,-23.11,9.38,NaN,NaN,NaN,NaN,-54380000.0,-152.73,NaN,NaN,NaN,11.95,NaN,NaN,0.72,0.00,-10.13,NaN,NaN,38110000.0


## Test Results
With minmax normalization: Test Score: 0.7628170815358735, RMSE: 29.983517
Without minmax normalization: Test Score: 0.8737420944473197, RMSE: 21.876113

In [ ]:
from company import *
company = Company("SNE")
model = xgb.XGBRegressor(objective='reg:squarederror')
model.load_model('xgb_main.model')
company.predict_price_using_xgb(model)